# Lab | Transformers

---

### Section structure

1. The open-source ecosystem: increasing accessibility to machine learning (ML) software and hardware
2. Some simple code demonstrations
3. Q&A

## 1. Ease-of-use: Using Transformers in 3 lines of code


**Overview of different tasks that can be automated with ML**
* Key ingredients: (1) a model trained on a specific task; (2) input data (e.g. texts or images); (3) output produced by the model.
* Transformers are currently the most popular type of deep learning algorithm. Most tasks below are solved with Transformers. There might be other types of algorithms coming up in the medium term.



**Install the Transformers library & dependencies**

In [ ]:
!pip install transformers  # The Transformers library from Hugging Face
!pip install sentencepiece
!pip install wikipedia
!pip install accelerate
!pip install tf-keras
!pip install torch

# NOTE: you might need to restart you jupyter kernel after installing the libraries

**The Hugging Face Pipeline**
* Makes automation of many NLP tasks possible in 3 lines of code
* Detailed documentation is available [here](https://huggingface.co/transformers/main_classes/pipelines.html)

In [2]:
from transformers import pipeline
import pandas as pd
import numpy as np
from pprint import pprint

Note : You might need more libraries or updates to run the cells below, if that is the case, follow the error messages and pip install accordingly. Chat gpt can help you if given the error messages.

### 2.1 Many models tailored to specific tasks


#### 2.1.1 Text classification

Let's select a popular text classification model in the [HF model hub](https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads).

Here we chose "cardiffnlp/twitter-roberta-base-irony".

We will classify text into ironic or non ironic.

In [ ]:
pipeline_classification = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-irony")  # cardiffnlp/twitter-roberta-base-irony, SamLowe/roberta-base-go_emotions

Now that we have the model we can pass it a string and have it give us a classification.

Feel free to experiment with different sentences by changing the contents of the variable text

In [ ]:
text = "Well that workshop was totally worth my time..."  # "Well that workshop was totally worth my time..."  "This smells weird, I'm not sure if I should eat this ... Yikes, it tasted like old socks!"
output = pipeline_classification(text, top_k=10)
print(output)

[{'label': 'irony', 'score': 0.9424387812614441}, {'label': 'non_irony', 'score': 0.057561274617910385}]


Let's make the output a little cleaner

In [ ]:
# make output a bit cleaner
df_output = pd.DataFrame(output)
print(df_output)

       label     score
0      irony  0.942439
1  non_irony  0.057561


As you can see, in a few lines of code and by leveraging an existing model we can classify text as ironic or non ironic. Now you have one more tool in your machine learning toolbox.

Remember that : 'when you only have a hammer everything is a nail'. But if we want to build a house (perform machine learning the right way), we need to use the right tool for the right job.

#### 2.1.2 Machine Translation

* Open source machine translation (MT) models enable you to translate between many different languages without Google Translate.
* [University of Helsinki](https://huggingface.co/Helsinki-NLP) uploaded models for more than 1000 language pairs to the Hugging Face hub
* [Facebook AI](https://huggingface.co/models?search=facebook+m2m) open-sourced several multi-lingual models
* The [EasyNMT library](https://github.com/UKPLab/EasyNMT), provides an easy wrapper for all these models
* Most machine translation models translate between two languages in one direction (e.g. German to English, but not English to German), some can translate in multiple directions.


In [23]:
# translation pipeline docs: https://huggingface.co/transformers/main_classes/pipelines.html#transformers.TranslationPipeline
pipeline_translate = pipeline("translation", model="facebook/m2m100_418M")

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Device set to use cuda:0


M2M100 is a multilingual encoder-decoder (seq-to-seq) model trained for Many-to-Many multilingual translation.

The model that can directly translate between the 9,900 directions of 100 languages.

Here we specify to translate from German 'de' to English 'en'

In [28]:
text = "Ich bin ein Fisch"
pipeline_translate(text, src_lang="de", tgt_lang="ar")

[{'translation_text': 'أنا سمكة'}]

Let's do the same but with and entire wikipedia page in german.

In [20]:
wikipedia.search("FC Bayern München")

['FC Bayern München',
 'FC Bayern München (Frauenfußball)',
 'FC Bayern München (Basketball)',
 'FC Bayern München AG',
 'FC Bayern München/Namen und Zahlen',
 'FC Bayern München/Europapokalstatistik',
 'Liste der Fußballspieler des FC Bayern München',
 'FC Bayern Campus',
 'Allianz Arena',
 'Fußball-Bundesliga']

In [38]:
# download any text from wikipedia, via  https://pypi.org/project/wikipedia/
import wikipedia
wikipedia.set_lang("es")

text = wikipedia.summary("Instituto Valenciano de Investigaciones Agrarias").replace('\n', ' ')[:318]
print(f"Original text:\n{text}\n")

# translate the text from wikipedia
text_translated = pipeline_translate(text, src_lang="es", tgt_lang="ar")
print(f"Translated text:\n{text_translated[0]['translation_text']}")


Original text:
El Instituto Valenciano de Investigaciones Agrarias (IVIA), es un Organismo Autónomo de la Generalidad Valenciana, adscrito a la Consejería de Agricultura, Pesca y Alimentación. Este instituto se encarga de promover y realizar proyectos, convenios o contratos de investigación en el sector agroalimentario.

Translated text:
معهد فالنسيا للبحوث الزراعية (IVIA) هو منظمة مستقلة في العاصمة فالنسيا، مخصصة لمجلس الزراعة والصيادين والأغذية.هذا المعهد هو المسؤول عن تعزيز وتنفيذ المشاريع أو الاتفاقيات أو العقود البحثية في قطاع الزراعة.


#### 2.1.3 Text Summarization

In [ ]:
# docs for summarisation pipeline: https://huggingface.co/transformers/main_classes/pipelines.html#summarizationpipeline
pipeline_summarize = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")  # sshleifer/distilbart-cnn-12-6 , google/pegasus-cnn_dailymail

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
# download any long text from wikipedia, via  https://pypi.org/project/wikipedia/
import wikipedia
wikipedia.set_lang("en")

text_long = wikipedia.summary("Donald Trump").replace('\n', ' ')
print(f"Original text:\n{text_long}\n")

# translate the text from wikipedia
text_summarized = pipeline_summarize(text_long, min_length=5, max_length=30)
print(f"Summarized text:\n{text_summarized[0]['summary_text']}")

Original text:
Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who served as the 45th president of the United States from 2017 to 2021. Trump received a Bachelor of Science in economics from the University of Pennsylvania in 1968. His father named him president of his real estate business in 1971. Trump renamed it the Trump Organization and reoriented the company toward building and renovating skyscrapers, hotels, casinos, and golf courses. After a series of business failures in the late 1990s, he launched successful side ventures, mostly licensing the Trump name. From 2004 to 2015, he co-produced and hosted the reality television series The Apprentice. He and his businesses have been plaintiffs or defendants in more than 4,000 legal actions, including six business bankruptcies. Trump won the 2016 presidential election as the Republican Party nominee against Democratic Party candidate Hillary Clinton while losing the popular vote. A 

#### 2.1.4 Named Entity Recognition

NER is a task that involves identifying and classifying specific entities in text into predefined categories, such as names of people, organizations, locations, dates, and more.

For example, in the sentence "Apple Inc. was founded by Steve Jobs in California," NER would recognize "Apple Inc." as an organization, "Steve Jobs" as a person, and "California" as a location.

In [ ]:
pipeline_ner = pipeline("token-classification", model="dslim/bert-base-NER-uncased", aggregation_strategy="simple")

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import wikipedia
wikipedia.set_lang("en")

text_long = wikipedia.summary("Donald Trump").replace('\n', ' ')

output = pipeline_ner(text_long)

pd.DataFrame(output)

,entity_group,score,word,start,end
0,PER,0.989843,donald john trump,0,17
1,MISC,0.991848,american,45,53
2,LOC,0.992675,united states,141,154
3,PER,0.989312,trump,174,179
4,ORG,0.697325,university of pennsylvania,233,259
5,PER,0.969404,trump,337,342
6,ORG,0.712140,trump organization,358,376
7,PER,0.913634,trump,597,602
8,MISC,0.953361,the apprentice,684,698
9,PER,0.992383,trump,828,833


### 2.2. Universal models

The models mentioned above are designed to excel at a single specific task on a particular dataset. The key advantage of these models is their high performance and accuracy on that specific task and dataset.

However, in real-world applications, the problems you'll face often require solving slightly different tasks, possibly with varied category definitions or applied to different types of texts.

Universal models can help address this challenge. Although they also focus on one task, the task is general or universal enough that many other tasks can be reformulated into it. Two examples of universal tasks are:

- Natural Language Inference (NLI): A task that can effectively solve a wide range of classification tasks by determining whether a given premise supports, contradicts, or is neutral with respect to a hypothesis.

- Token Generation: An even more universal task that can be applied to solve virtually any text-related task, including translation, summarization, and text completion.

These universal tasks enable the models to be versatile and adaptable to various problems beyond the specific ones they were initially trained on.

#### Zero-shot classification


Zero-shot classification is a technique where a model can categorize data into classes it has never seen before.

Instead of relying on labeled examples for each class, the model understands the relationship between the input and the class descriptions, allowing it to make accurate predictions without needing specific training on those classes.

In [ ]:
pipeline_zeroshot_classification = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Here we will give the model a list of classes ('payment issues', 'travel advice', 'bug report') for it to classify our string.

In [ ]:
text = "Customer: I have not received my reimbursement yet. What the hell is going on?"
classes = ['payment issues', 'travel advice', 'bug report']  # "account opening", "customer complaint"

#text = "I do not think the government is trustworthy anymore. We need to mobilize and resist!"
#classes = ["civil disobedience", "praise of the government", "travel advice"]  # "collective action"

output = pipeline_zeroshot_classification(text, classes, multi_label=True)

pd.DataFrame(data=[output["labels"], output["scores"]], index=["class", "probability"]).T


,class,probability
0,payment issues,0.991133
1,bug report,0.076115
2,travel advice,0.018696


## Exercise

Now it is your turn to go to the hugging face library https://huggingface.co/models?pipeline_tag=text-classification&sort=downloads

(you can select on the left menu of the website the type of NLP tasks you want models to perform)

- Find an NLP model that we have not used previously.

- Get some data from wikipedia or elsewhere.

- Perform inference with the model and print the result!

- Comment your code along the way, describe what your model does and what your end goal is from input to output.

Have fun!

## #1 Image to Text

In [3]:
# Your code here :
# Use a pipeline as a high-level helper
from transformers import pipeline
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [4]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

In [5]:
# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

a woman sitting on the beach with her dog


## #2 sentence-transformers

In [ ]:
pip install -U sentence-transformers

In [37]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 6.76568747e-02  6.34958893e-02  4.87130918e-02  7.93049634e-02
   3.74480635e-02  2.65279156e-03  3.93749624e-02 -7.09843310e-03
   5.93614280e-02  3.15370113e-02  6.00980744e-02 -5.29051796e-02
   4.06067744e-02 -2.59308126e-02  2.98428331e-02  1.12690555e-03
   7.35148862e-02 -5.03819399e-02 -1.22386619e-01  2.37028506e-02
   2.97265127e-02  4.24768664e-02  2.56337617e-02  1.99516979e-03
  -5.69191203e-02 -2.71598138e-02 -3.29035893e-02  6.60248473e-02
   1.19007148e-01 -4.58791256e-02 -7.26214871e-02 -3.25839370e-02
   5.23414202e-02  4.50552851e-02  8.25298578e-03  3.67023796e-02
  -1.39415348e-02  6.53918907e-02 -2.64272615e-02  2.06381184e-04
  -1.36643257e-02 -3.62809934e-02 -1.95043553e-02 -2.89738476e-02
   3.94270271e-02 -8.84090886e-02  2.62425840e-03  1.36713935e-02
   4.83063273e-02 -3.11566070e-02 -1.17329195e-01 -5.11690341e-02
  -8.85287896e-02 -2.18962487e-02  1.42986784e-02  4.44167703e-02
  -1.34815173e-02  7.43392482e-02  2.66382713e-02 -1.98762342e-02
   1.79191

In [38]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)

torch.Size([4, 4])
